In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,LogisticRegression
ruta='/content/drive/MyDrive/Colab Notebooks/ITESO/Ingeniería de características/P2022_MCD3735A/Módulo 4 - Codificación de variables categóricas/'

data=pd.read_csv(ruta+'temps.txt',sep='\s+',header=None) # Fuente: http://astro.iam.udg.mx/Estacion/NOAAPRMO.TXT
                                                         #         http://astro.iam.udg.mx/Estacion/NOAAMO.TXT
data.columns=['Temperatura máxima','Temperatura mínima','HDD','CDD','Rain']
d=['Sábado','Domingo','Lunes','Martes','Miércoles','Jueves','Viernes']
dias=[d[i%7] for i in range(len(data))]
catego=['Soleado','Mayormente soleado','Parcialmente nublado','Mayormente nublado','Nublado','Lluvioso']
prono=[catego[-1] if data['Rain'][i]>0 else catego[int(round(1.2*(data['HDD'][i]-data['CDD'][i])+2))] for i in range(len(data))]
df=pd.DataFrame(columns=['Días','Pronóstico','Temperatura máxima','Temperatura mínima'],index=range(len(data)))
df.iloc[:,2:4]=data.iloc[:,0:2]
df['Días']=dias
df['Pronóstico']=prono
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima
0,Sábado,Parcialmente nublado,28.2,11.2
1,Domingo,Parcialmente nublado,29.1,10.7
2,Lunes,Mayormente nublado,26.1,10.4
3,Martes,Mayormente soleado,29.4,11.1
4,Miércoles,Mayormente soleado,29.8,11.1


# Codificación usando el promedio de `Temperatura máxima`

In [6]:
promedios=df.groupby('Pronóstico')['Temperatura máxima'].mean()
promedios

Pronóstico
Lluvioso                27.250000
Mayormente nublado      25.600000
Mayormente soleado      28.228571
Nublado                 25.650000
Parcialmente nublado    27.022222
Soleado                 28.900000
Name: Temperatura máxima, dtype: float64

In [7]:
pm=promedios.to_dict()
pm

{'Lluvioso': 27.25,
 'Mayormente nublado': 25.599999999999998,
 'Mayormente soleado': 28.228571428571428,
 'Nublado': 25.65,
 'Parcialmente nublado': 27.022222222222222,
 'Soleado': 28.899999999999995}

In [8]:
dfC=df.copy()
df['Pronóstico media']=df['Pronóstico'].map(pm)
df.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico media
0,Sábado,Parcialmente nublado,28.2,11.2,27.022222
1,Domingo,Parcialmente nublado,29.1,10.7,27.022222
2,Lunes,Mayormente nublado,26.1,10.4,25.600000
3,Martes,Mayormente soleado,29.4,11.1,28.228571
4,Miércoles,Mayormente soleado,29.8,11.1,28.228571


# Usando la mediana de `Temperatura máxima`

In [10]:
pme=df.groupby('Pronóstico')['Temperatura máxima'].median().to_dict()
pme

{'Lluvioso': 27.25,
 'Mayormente nublado': 26.15,
 'Mayormente soleado': 28.450000000000003,
 'Nublado': 25.65,
 'Parcialmente nublado': 27.0,
 'Soleado': 28.799999999999997}

In [11]:
dfC['Pronóstico mediana']=df['Pronóstico'].map(pme)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico mediana
0,Sábado,Parcialmente nublado,28.2,11.2,27.00
1,Domingo,Parcialmente nublado,29.1,10.7,27.00
2,Lunes,Mayormente nublado,26.1,10.4,26.15
3,Martes,Mayormente soleado,29.4,11.1,28.45
4,Miércoles,Mayormente soleado,29.8,11.1,28.45


# Codificación a partir de Efectos

In [31]:
F=pd.get_dummies(df['Pronóstico'],drop_first=True,prefix='',prefix_sep='')
F.head()

,Mayormente nublado,Mayormente soleado,Nublado,Parcialmente nublado,Soleado
0,0,0,0,1,0
1,0,0,0,1,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


In [42]:
set(df['Pronóstico'].unique())-set(F.columns)

{'Lluvioso'}

In [33]:
E=F.astype('int8')
E.iloc[np.where(df['Pronóstico']=='Lluvioso')[0]]=-1
E.iloc[17]

Mayormente nublado     -1
Mayormente soleado     -1
Nublado                -1
Parcialmente nublado   -1
Soleado                -1
Name: 17, dtype: int8

In [34]:
E.head()

,Mayormente nublado,Mayormente soleado,Nublado,Parcialmente nublado,Soleado
0,0,0,0,1,0
1,0,0,0,1,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0


In [35]:
lin=LinearRegression()
lin.fit(E,df['Temperatura máxima'])
lin.coef_,lin.intercept_

(array([-1.50846561,  1.12010582, -1.45846561, -0.08624339,  1.79153439]),
 27.10846560846561)

In [36]:
pe={c:i for c,i in zip(E.columns,lin.coef_)}
pe

{'Mayormente nublado': -1.5084656084656072,
 'Mayormente soleado': 1.1201058201058205,
 'Nublado': -1.4584656084656085,
 'Parcialmente nublado': -0.08624338624338601,
 'Soleado': 1.7915343915343904}

In [37]:
pe['Lluvioso']=lin.intercept_
dfC['Pronóstico Efectos']=df['Pronóstico'].map(pe)
dfC.head()

,Días,Pronóstico,Temperatura máxima,Temperatura mínima,Pronóstico mediana,Pronóstico Efectos
0,Sábado,Parcialmente nublado,28.2,11.2,27.00,-0.086243
1,Domingo,Parcialmente nublado,29.1,10.7,27.00,-0.086243
2,Lunes,Mayormente nublado,26.1,10.4,26.15,-1.508466
3,Martes,Mayormente soleado,29.4,11.1,28.45,1.120106
4,Miércoles,Mayormente soleado,29.8,11.1,28.45,1.120106


In [38]:
dfC.iloc[17]

Días                     Martes
Pronóstico             Lluvioso
Temperatura máxima         27.9
Temperatura mínima         11.5
Pronóstico mediana        27.25
Pronóstico Efectos    27.108466
Name: 17, dtype: object